In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torchvision
import torch.nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 랜덤 시드 고정
torch.manual_seed(777)
# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
import pandas as pd

file_path3 = '/content/gdrive/MyDrive/MAIC2023_공모전/submission.csv'
submission = pd.read_csv(file_path3)

In [ ]:
import os
import io
import zipfile
from zipfile import ZipFile
from io import BytesIO
from numpy.lib.type_check import common_type
import numpy as np

cv_filename = []  ##파일이름 리스트
cv_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_valid.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            cv_data.append(curr_df)
            cv_filename.append(fileName)


av_filename = []  ##파일이름 리스트
av_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_valid.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            av_data.append(curr_df)
            av_filename.append(fileName)

In [ ]:
for i in range(len(cv_data)):
    cv_data[i] = cv_data[i].reshape(5000, 12)

for i in range(len(av_data)):
    av_data[i] = av_data[i].reshape(5000, 12)

In [ ]:
new_list2 = [item.replace('.npy', '') for item in cv_filename]
new_list3 = [item.replace('.npy', '') for item in av_filename]

In [ ]:
combined_data_child = pd.DataFrame({'FILENAME': new_list2, 'ECG_SIGNAL': cv_data})
combined_data_adult = pd.DataFrame({'FILENAME': new_list3, 'ECG_SIGNAL': av_data})
combined_valid = pd.concat([combined_data_child, combined_data_adult], axis=0)
combined_valid = pd.merge(submission, combined_valid, on='FILENAME', how='inner')
combined_valid = combined_valid[combined_valid['GENDER'] != 'unknown']
combined_valid["GENDER"] = combined_valid['GENDER'].map({'MALE' :0, 'FEMALE':1})
combined_valid

,FILENAME,GENDER,AGE,ECG_SIGNAL
0,ecg_child_8781,1,NaN,"[[-14.0, -14.0, -14.0, -14.0, -7.0, 0.0, 7.0, ..."
1,ecg_child_8782,0,NaN,"[[-8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8..."
2,ecg_child_8783,0,NaN,"[[-30.0, -30.0, -30.0, -30.0, -25.0, -22.0, -1..."
3,ecg_child_8784,1,NaN,"[[-6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6..."
4,ecg_child_8785,1,NaN,"[[-64.0, -64.0, -63.0, -63.0, -59.0, -56.0, -5..."
...,...,...,...,...
5470,ecg_adult_39536,1,NaN,"[[-8.0, -8.0, -8.0, -8.0, -6.0, -6.0, -6.0, -6..."
5471,ecg_adult_39537,1,NaN,"[[2.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0,..."
5472,ecg_adult_39538,1,NaN,"[[26.0, 26.0, 26.0, 26.0, 27.0, 28.0, 29.0, 30..."
5473,ecg_adult_39539,1,NaN,"[[-15.0, -9.0, -1.0, 5.0, 2.0, -4.0, -2.0, -5...."


In [ ]:
'''
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


scaler = MinMaxScaler()
# 모델 로드
loaded_model = load_model("/content/gdrive/MyDrive/MAIC2023_공모전/my_model.h5")

# combined_valid 데이터의 'ECG_SIGNAL' 열을 넘파이 배열로 변환
#combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(eval)

# combined_valid 데이터의 'ECG_SIGNAL' 데이터 정규화
combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

# combined_valid 데이터를 시계열 데이터로 변환
X_submission = np.array(combined_valid['ECG_SIGNAL'].tolist())

# AGE 예측
predicted_age = loaded_model.predict(X_submission)

# 예측 결과를 'AGE' 컬럼에 채우기
combined_valid['AGE'] = predicted_age

# 결과 저장
combined_valid.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age.csv', index=False)
'''

'\nfrom tensorflow.keras.models import load_model\nfrom sklearn.preprocessing import MinMaxScaler\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import LSTM, Dense\nfrom tensorflow.keras.optimizers import Adam\n\n\nscaler = MinMaxScaler()\n# 모델 로드\nloaded_model = load_model("/content/gdrive/MyDrive/MAIC2023_공모전/my_model.h5")\n\n# combined_valid 데이터의 \'ECG_SIGNAL\' 열을 넘파이 배열로 변환\n#combined_valid[\'ECG_SIGNAL\'] = combined_valid[\'ECG_SIGNAL\'].apply(eval)\n\n# combined_valid 데이터의 \'ECG_SIGNAL\' 데이터 정규화\ncombined_valid[\'ECG_SIGNAL\'] = combined_valid[\'ECG_SIGNAL\'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))\n\n# combined_valid 데이터를 시계열 데이터로 변환\nX_submission = np.array(combined_valid[\'ECG_SIGNAL\'].tolist())\n\n# AGE 예측\npredicted_age = loaded_model.predict(X_submission)\n\n# 예측 결과를 \'AGE\' 컬럼에 채우기\ncombined_valid[\'AGE\'] = predicted_age\n\n# 결과 저장\ncombined_valid.to_csv(\'/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age.cs

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


scaler = MinMaxScaler()
# 모델 로드
loaded_model = load_model("/content/gdrive/MyDrive/MAIC2023_공모전/my_model_adult.h5")

# combined_valid 데이터의 'ECG_SIGNAL' 열을 넘파이 배열로 변환
#combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(eval)

# combined_valid 데이터의 'ECG_SIGNAL' 데이터 정규화
combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

# combined_valid 데이터를 시계열 데이터로 변환
X_submission = np.array(combined_valid['ECG_SIGNAL'].tolist())

# AGE 예측
predicted_age = loaded_model.predict(X_submission)

# 예측 결과를 'AGE' 컬럼에 채우기
combined_valid['AGE'] = predicted_age

# 결과 저장
combined_valid.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age_adult.csv', index=False)

171/171 [==============================] - 98s 550ms/step


In [ ]:
import pandas as pd

file_path2 = '/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age.csv'
file_path2 = '/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age_adult.csv'

